In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import animation, rc

from ex1.main import Simulation
from ex1.main import read_scenario
from ex1.visualizer import Visualizer

rc('animation', html='jshtml')
%matplotlib inline

In [ ]:
scenario = read_scenario('scenario_task_4.json')
states = Simulation(scenario).get_states()

anim = Visualizer(states).draw()
anim

In [ ]:
#vis.get_video(anim)